🏠 House Price Prediction Project

This project focuses on predicting the selling price of residential homes using machine learning techniques. The dataset contains various features such as overall quality, living area, garage size, year built, and many others that influence house prices.

The goal is to build a model that learns relationships between house features and their sale prices and then uses it to predict prices for unseen data.


🧩 Problem Statement

The objective of this project is to:

- analyze real estate housing data  
- handle missing and inconsistent values  
- build machine learning models to predict house sale prices  
- evaluate model accuracy  
- generate predictions for the test dataset

The target variable is:

```
SalePrice
```

---

Importing the Dependencies

In [ ]:
import pandas as pd # data manipulation and analysis
import numpy as np # numerical computing
from scipy import stats 
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.kernel_ridge import KernelRidge

In [ ]:
# Install catboost in the notebook environment
%pip install catboost

In [ ]:
# load the dataset
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
train_df.set_index("Id", inplace=True)  # type: ignore 
test_df.set_index("Id", inplace=True)   # type: ignore

print("Train dataset shape:", train_df.shape) # type: ignore
print("Test dataset shape:", test_df.shape) # type: ignore

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
train_df.isnull().sum()

Handelinng Outliers

In [ ]:
plt.figure(figsize=(14, 6))
sns.histplot(train_df['SalePrice'], bins=50, kde=True)
plt.title('Distribution of SalePrice')
plt.xlabel('SalePrice')
plt.ylabel('Frequency')
plt.show()

print(f"Skewness of sale price:{train_df.SalePrice.skew()}")

In [ ]:
train_df['SalePrice'] = np.log1p(train_df['SalePrice'])

In [ ]:
plt.figure(figsize=(14, 6))
sns.histplot(train_df['SalePrice'], bins=50, kde=True)
plt.title('Distribution of SalePrice')
plt.xlabel('SalePrice')
plt.ylabel('Frequency')
plt.show()

print(f"Skewness of sale price:{train_df.SalePrice.skew()}")

In [ ]:
corrmat = train_df.corr(numeric_only=True)
top_corr_features = corrmat.nlargest(15,'SalePrice')['SalePrice'].index
top_corr_matrix = train_df[top_corr_features].corr()
top_corr_matrix

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(top_corr_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix of top 10 features with SalePrice")
plt.show()

✅ Most strongly correlated with SalePrice (top 5–6 features):

OverallQual (0.82) – Overall quality of the house

GrLivArea (0.70) – Above ground living area in square feet

GarageCars (0.68) – Number of cars the garage can hold

GarageArea (0.65) – Size of garage in square feet

TotalBsmtSF (0.61) – Total basement area

1stFlrSF (0.60) – First floor area

These features are likely the best predictors for SalePrice in a model.

🧹 Data Preprocessing

The following preprocessing steps were performed:

- handled missing values (mean/median/mode depending on feature type)
- encoded categorical columns into numerical format
- treated outliers in important features
- performed feature scaling where required
- applied log transformation on SalePrice to reduce skewness
- split data into training and validation sets

These steps ensure the dataset is clean and suitable for model training.

---


In [ ]:
#Combine test and train data
data = pd.concat((train_df.loc[:,:'SaleCondition'], test_df.loc[:, :'SaleCondition']))

print(f"Combined data shape: {data.shape}")

In [ ]:
data.isnull().sum()

In [ ]:
missing_data = data.isnull().sum().sort_values(ascending=False) # Count missing values in each column
missing_data = missing_data[missing_data > 0] # Filter columns with missing values 
missing_data # Display missing data information


In [ ]:
missing_data.shape

In [ ]:
# I want to count how many columns have missing values less than 100
print(missing_data[missing_data < 100])
missing_data[missing_data < 100].shape

In [ ]:
# which collumns have only few missing values they can be filled with mean/median/mode

zero_fill_cols = [
    'MasVnrArea',
    'BsmtFinSF1',
    'BsmtFinSF2',
    'BsmtUnfSF',
    'TotalBsmtSF',
    'GarageCars',
    'GarageArea',
    'BsmtFullBath',
    'BsmtHalfBath'
]
for col in zero_fill_cols:
    if col in data.columns:
        data[col] = data[col].fillna(0)
# Impute numerical features with 0

In [ ]:
median_fill_cols = [
    'GrLivArea',
    '1stFlrSF'
]

for col in median_fill_cols:
    if col in data.columns:
        data[col] = data[col].fillna(data[col].median())
# Impute numerical features with median

In [ ]:
mode_fill_cols = [
    'Electrical',
    'KitchenQual',
    'Exterior1st',
    'MSZoning',
    'SaleType'
]

for col in mode_fill_cols:
    if col in data.columns:
        data[col] = data[col].fillna(data[col].mode()[0])
        

# Impute categorical features with mode

In [ ]:
data.isnull().sum()[data.isnull().sum() > 0]


In [ ]:
data['LotFrontage'].isnull().sum()

In [ ]:
if 'LotFrontage' in data.columns and data['LotFrontage'].isnull().any():
    data['LotFrontage'] = data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

In [ ]:
data['LotFrontage'].isnull().sum()

In [ ]:
# Basement related 
bsmt_cols = [
    'BsmtQual', 'BsmtCond', 'BsmtExposure',
    'BsmtFinType1', 'BsmtFinType2'
]

for col in bsmt_cols:
    data[col] = data[col].fillna('None')


In [ ]:
# Garage related
garage_cat_cols = [
    'GarageType', 'GarageFinish',
    'GarageQual', 'GarageCond'
]

for col in garage_cat_cols:
    data[col] = data[col].fillna('None')


In [ ]:
# Other categorical features
none_fill_cols = [
    'Alley', 'MasVnrType', 'FireplaceQu',
    'PoolQC', 'Fence', 'MiscFeature'
]

for col in none_fill_cols:
    data[col] = data[col].fillna('None')


In [ ]:
# GarageYrBlt 
data['GarageYrBlt'] = data['GarageYrBlt'].fillna(0)


In [ ]:
data['Utilities'] = data['Utilities'].fillna(data['Utilities'].mode()[0])


In [ ]:
data['Exterior2nd'] = data['Exterior2nd'].fillna(data['Exterior2nd'].mode()[0])


In [ ]:
data['Functional'] = data['Functional'].fillna('Typ')


In [ ]:
print("\nMissing values after all imputation:", data.isnull().sum().sum())

In [ ]:
# ##Feature Engineering
# - Need to combine square footage
# - Need to combine the bathrooms
# - Need to create a feature for age oh house at sale

In [ ]:
# Need to combine square footage
data['TotalSF'] = (
    data['TotalBsmtSF'] +
    data['1stFlrSF'] +
    data['2ndFlrSF']
)
# Need to combine the bathrooms 
data['Total_Bathrooms'] = (
    data['FullBath'] +
    (0.5 * data['HalfBath']) +
    data['BsmtFullBath'] +
    (0.5 * data['BsmtHalfBath'])
)
# - Need to create a feature for age oh house at sale
data['House_Age'] = data['YrSold'] - data['YearBuilt']
print("Feature engineering completed.")

In [ ]:
##Categorical encoding

In [ ]:
data['Neighborhood'].value_counts().plot(kind='bar', figsize=(12,5))

In [ ]:
#one-hot encoding

In [ ]:
print(f"Data shape before encoding: {data.shape}")
categorical_cols = data.select_dtypes(include=['object']).columns # select categorical columns
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True) # Converts each categorical column into binary (0/1) columns
print(f"Data shape after encoding: {data.shape}")

🤖 Model Training

The following models were trained:

- Linear Regression
- XGBoost Regressor

Training steps:

1. Fit the model on the training dataset  
2. Validate using the validation dataset  
3. Tune hyperparameters to improve performance  

For XGBoost, techniques such as early stopping and regularization were applied to prevent overfitting.

---


In [ ]:
# Split back into train and test sets
train_data = data.loc[train_df.index]
test_data = data.loc[test_df.index]

# Note: 'SalePrice' was not included when combining data, so use original train_df for the target
X = train_data  # features (already does not contain SalePrice)
y = train_df['SalePrice']  # target variable from original train_df

# Split into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(test_data)

In [ ]:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)


In [ ]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

In [ ]:
y_pred_lr = lr.predict(X_val_scaled)
y_pred_lr

In [ ]:
comparison = pd.DataFrame({
    'Actual': y_valid,
    'Predicted': y_pred_lr
})
comparison.head()


📈 Evaluation

The model performance was evaluated using:

- RMSE (Root Mean Squared Error)
- R² Score
- Actual vs Predicted comparison

Log-transformed predictions were converted back using:

```
np.expm1()
```

Lower RMSE and higher R² indicate a better model.

---

In [ ]:
# ##XGBoost

# - Gradient boosting algorithm
# - Builds trees sequentially. Each new tree attempts to correct the errors made by the previous trees. The predictions from all the trees are then summed up to get the final prediction.
# # - Known for its performance on structured data

# Use already imported XGBRegressor from earlier cell and the scaled datasets
from xgboost import XGBRegressor

xgb_model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=3,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror',
    random_state=42
)

# Fit without early stopping
xgb_model.fit(X_train_scaled, y_train)

# Predict
y_pred = xgb_model.predict(X_val_scaled)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_model(y_true, y_pred, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"--- {model_name} Performance ---")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE:  {mae:.4f}")
    print(f"R-squared: {r2:.4f}\n")

# Linear Regression evaluation
evaluate_model(y_valid, y_pred_lr, "Linear Regression")

# XGBoost evaluation
evaluate_model(y_valid, y_pred, "XGBoost")


In [ ]:
import pandas as pd

metrics = {
    "Model": ["Linear Regression", "XGBoost"],
    "RMSE": [
        np.sqrt(mean_squared_error(y_valid, y_pred_lr)),
        np.sqrt(mean_squared_error(y_valid, y_pred))
    ],
    "MAE": [
        mean_absolute_error(y_valid, y_pred_lr),
        mean_absolute_error(y_valid, y_pred)
    ],
    "R²": [
        r2_score(y_valid, y_pred_lr),
        r2_score(y_valid, y_pred)
    ]
}

metrics_df = pd.DataFrame(metrics)
print(metrics_df)


In [ ]:
# i want to check accuracy of test data in percentage
from sklearn.metrics import r2_score
r2_lr = r2_score(y_valid, y_pred_lr)
r2_xgb = r2_score(y_valid, y_pred)

In [ ]:
# Use XGBoost to predict
# model was trained on scaled features, so use X_test_scaled
y_test_pred_log = xgb_model.predict(X_test_scaled)
# invert the earlier log1p transform
y_test_pred = np.expm1(y_test_pred_log)

submission = pd.DataFrame({
    "Id": test_df.index,
    "SalePrice": y_test_pred
})
submission.to_csv("submission.csv", index=False)
print("Submission file created successfully!")

Conclusion

- Machine learning models can effectively predict house prices
- Feature engineering and preprocessing significantly improve accuracy
- XGBoost performed better than simple linear regression
- The model can be used for real-estate price estimation and decision-making

Future improvements:

- try more ensemble models
- perform feature selection
- hyperparameter tuning with GridSearch
- deploy model using Flask/Streamlit